**This script reads the scene_gt_info.json, scene_gt.json from BOP Dataset format and rgb.txt created using rgb_txt_generator.ipynb. It will convert this format into the bounding box format that is needed for the OA-SLAM detections_yolov5.json**

In [4]:
#### IMPORT MODULES ####
import json
import glob
import os

In [5]:
### PATH TO DATASET ###
datasets = ["/home/allen/Desktop/RnD_Github/AllenIsaacRnD/noisy_bounding_box_experiment/noisy_scene"]

In [6]:
for data_path in datasets:

    ### JSON files in BOP format are to be accessed ###
    scene_gt_info_path = data_path + '/scene_gt_info.json'

    ### rgb.txt file which is created using rgb_txt_generator.ipynb
    RGB_txt_path = data_path + '/rgb/rgb.txt'

    ### output file that is generated
    output_file = data_path + '/detections_yolov5.json'
    
    
    
    ### Access the name of the images from rgb.txt
    with open(RGB_txt_path, 'r') as file:
        lines = file.readlines()

    images = []
    # Access the data from the text file
    for line in lines:
        # Perform operations on each line of data
        # For example, printing the content:
        images.append(line.strip())
        
    # Read the JSON file
    with open(scene_gt_info_path, 'r') as f:
        scene_gt_info_data = json.load(f)
 

    output_json = [] ## data that is to be appended into the json file


    # key is image name and value is the bounding box info of each object
    for idx in range(len(scene_gt_info_data)):
        temp1 = dict()
        key = str(int(images[idx].split(".")[0])) ## get int of image name which is the key
        value = scene_gt_info_data[key]
        temp1['file_name'] = images[idx]
        temp1['detections'] = []

        # iterate through each object in the image
        for o in range(len(value)):
            temp2 = dict()
            # get the object class id
            temp2['category_id'] = int(value[o]['class_label'])
            # since detection score is not present in BOP dataset,
            # we take it as visible fraction
            #temp2['detection_score'] = value[o]['visib_fract'] ## changed for current need
            temp2['detection_score'] = 0.95

            # need to check if the bounding box is correct
            # (x, y, width, height) is what is given in BOP dataset where (x,y) is top left corner
            # temp2['bbox'] = value[o]['bbox_obj'] # try 'bbox_visib' also
            # (x1, y1, x2, y2) for detectron, the left top corner and right bottom corner
            temp2['bbox'] = value[o]['bbox_obj']
            temp2['bbox'][2] += temp2['bbox'][0]
            temp2['bbox'][3] += temp2['bbox'][1]

            temp1['detections'].append(temp2)


        output_json.append(temp1)
        
    ### Save the list of data as a JSON file
    with open(output_file, 'w') as file:
        json.dump(output_json, file)

    print('JSON file created successfully.')

JSON file created successfully.
